In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tensorflow as tf

# needed to avoid a tf error
try:
    gpus= tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)
except:
    pass

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
import numpy as np

# Boston Model

In [2]:
# loading models that we prepared earlier
X_train = np.load(os.path.join('data', 'boston', 'boston_train_vectors.npy'), allow_pickle=True)
X_test = np.load(os.path.join('data', 'boston', 'boston_test_vectors.npy'), allow_pickle=True)
y_train = np.load(os.path.join('data', 'boston', 'boston_train_labels.npy'), allow_pickle=True)
y_test = np.load(os.path.join('data', 'boston', 'boston_test_labels.npy'), allow_pickle=True)

In [3]:
X_train.shape, y_train.shape

((404, 13), (404,))

In [4]:
# setting seed to get reproducible results
tf.random.set_seed(42)
np.random.seed(42)

# building a small model as an experiment
model = Sequential()
model.add(Dense(13, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='sgd', metrics=['mae'])

In [5]:
# early stopping is not used in the current implementation, but we plan to use it in the final model
cb = [EarlyStopping(monitor="val_mae", min_delta=0.01, patience=2, verbose=1, 
                    mode="auto", baseline=None, restore_best_weights=True)]

In [6]:
#model.fit(X_train, y_train, batch_size=1, epochs=20, validation_split=0.1, callbacks=cb)

# Using the custom training loop

In [7]:
# importing our custom loop
from loop import TrainingLoop

In [8]:
# using SGD oprimizer for training
optimizer = tf.keras.optimizers.SGD()

# MSE loss function for this regression task
loss_function = tf.keras.losses.MeanSquaredError()

batch_size = 8
epochs = 20

# using MAE as our secondary metric
train_metrics = tf.keras.metrics.MeanAbsoluteError()
val_metrics = tf.keras.metrics.MeanAbsoluteError()

In [9]:
@tf.function
def calc_loss(x_train, y_train):
    with tf.GradientTape() as tape:
        logits = model(x_train, training=False)
        loss_value = loss_function(y_train, logits)
    return loss_value


def batch_selector(data, idx):
    largest_loss = 0
    largest_loss_idx = idx

    if idx < len(data) - 5:
        for i in range(idx, idx+5):
            x_batch_train = data[i][0]
            y_batch_train = data[i][1]
            loss = calc_loss(x_batch_train, y_batch_train)
            if loss > largest_loss:
                largest_loss = loss
                largest_loss_idx = i
        return largest_loss_idx
    else:
        loss = calc_loss(data[idx][0], data[idx][1])
        return idx

In [10]:
# defininf the training class
training = TrainingLoop(model, X_train, y_train, 
                        loss_function, 
                        optimizer, 
                        train_metrics, 
                        val_metrics, 
                        validation_split=0.1, 
                        batch_size=batch_size,
                        batch_selection=batch_selector,
                        length=5
                        )

In [11]:
training.train(epochs)

	  0% | 0/45 [00:00<?, ?it/s]WARNING:tensorflow:Layer dense is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because its dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/20	Loss: 19.631	Metrics: 9.3871: 	Validation metrics: 6.7541: 	100% | 45/45 [00:00<00:00, 66.93it/s]
Epoch 2/20	Loss: 5.8102	Metrics: 7.7742: 	Validation metrics: 3.7531: 	100% | 45/45 [00:00<00:00, 224.48it/s]
Epoch 3/20	Loss: 6.7007	Metrics: 5.5165: 	Va

In [12]:
res = model.evaluate(X_test, y_test)
print(np.sqrt(res[0]), res[1])

4/4 [==============================] - 0s 2ms/step - loss: 11.6431 - mae: 2.6134
3.412192244852643 2.6133832931518555
